In this Notebook, we will train a CNN to solve the chinese MNIST dataset

I - Imports & Installs

In [ ]:
!pip install tqdm

In [84]:
from tqdm import tqdm
import matplotlib as plt
import copy 

%run CNN_layers.ipynb
%run Chinese_MNIST_preprocessing.ipynb

# np.random.seed(0)  # For reproductibility

Skipping, found downloaded files in ".\chinese-mnist" (use force=True to force download)


II - Neural network

In [87]:
for e in tqdm(range(epochs)) :

    error = 0
    
    batch1_copy = copy.deepcopy(batch1)

    for nb_minibatch in range(66) :

        minibatch = []   # minibatches of size 45

        indexes = []

        for nb in range(15) : 
            indices_binned = np.random.choice(198 - 3*nb_minibatch, size = 3, replace = 'False')
            for i in range(3) :
                minibatch.append(batch1_copy[indices_binned[i] + (198 - 3*nb_minibatch)*nb])
                indexes.append(indices_binned[i] + (198 - 3*nb_minibatch)*nb)
        
        i = 0 
        for j in range(len(indexes)) :
            ind = indexes[j] 
            del(batch1_copy[ind - i])
            i+=1
            
        batch1_copy.sort(key = lambda x : int(x[:][1][-2:]))

        np.random.shuffle(minibatch)

        minibatch = preprocess(minibatch)

        grad = np.zeros((15,))

        for x,y in zip(minibatch[0],minibatch[1]) :
        
            # Forward propagation :
            output = x
    
            for layer in image_bluring_layers : 
                output = layer.forward(output)

            for layer in model :
                output = layer.forward(output)
        
            loss = cross_entropy(output,y)
            error += loss.compute()
            grad += loss.grad()


        # Backward propagation

        grad /= 3

        for layer in reversed(model) :          
            grad = layer.backward(grad,eta)

    
    error /= len(batch1)
    print(f"mean epoch {e+1} error : ", str(error))

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:13<00:55, 13.98s/it]

mean epoch 1 error :  2.7684616306884617


 40%|████      | 2/5 [00:27<00:41, 13.79s/it]

mean epoch 2 error :  2.7684470964247545


 60%|██████    | 3/5 [00:42<00:28, 14.12s/it]

mean epoch 3 error :  2.7724754497521222


 60%|██████    | 3/5 [01:00<00:40, 20.21s/it]


IndexError: list assignment index out of range

In [ ]:
image_bluring_layers = [avg_pool_layer((1,64 - 2*i,64 - 2*i),3) for i in range(3)]


model = [
    convolutionnal_layer((1,58,58),6,1, optimizer = "moment_GD", moment_gd_param = 0.3),
    avg_pool_layer((1,53,53),3),
    ReLU_activation_layer(),
    reshape_layer((1,51,51),1*51*51),
    dense_layer(1*51*51,100,optimizer = "moment_GD", moment_gd_param = 0.3),
    normalize_layer1D(100),
    sigmoid_activation_layer(),
    dense_layer(100,15, optimizer = "moment_GD", moment_gd_param = 0.3),
    normalize_layer1D(15),
    sigmoid_activation_layer(),
    softmax_activation_layer()
]

# # Hyper parameters

epochs = 5
eta = 0.001

III - Training

1st batch

In [304]:


for e in tqdm(range(epochs)) :

    error = 0

    for x,y in zip(preprocess(batch1,len(batch1))[0],preprocess(batch1,len(batch1))[1]) :
        
        # Forward propagation :
        output = x
    
        for layer in image_bluring_layers : 
            output = layer.forward(output)

        for layer in model :
            output = layer.forward(output)
        
        loss = cross_entropy(output,y)
        error += loss.compute()
    
        # Backward propagation

        grad = loss.grad() 

        for layer in reversed(model) :          
            grad = layer.backward(grad,eta)

        # print(f"conv_layer 1 kernel --  mean : {np.mean(model[0].kernels)}", f" --  std : {np.std(model[0].kernels)}")
        # print(f"conv_layer 2 kernel -- mean : {np.mean(model[2].kernels)}", f" -- std  : {np.std(model[2].kernels)}")
        # print(f"dense_layer 1 weights -- mean : {np.mean(model[5].weights)}", f" -- std : {np.std(model[5].weights)}")
        # print(f"dense_layer 1 weights -- mean : {np.mean(model[8].weights)}", f" -- std : {np.std(model[8].weights)}")
        # print(" ")

    error /= len(batch1)
    print(f"mean epoch {e+1} error : ", str(error))    


 20%|██        | 1/5 [03:36<14:25, 216.43s/it]

mean epoch 1 error :  2.804896763366061


 40%|████      | 2/5 [07:52<11:24, 228.31s/it]

mean epoch 2 error :  2.804700577455325


 60%|██████    | 3/5 [10:51<07:06, 213.47s/it]

mean epoch 3 error :  2.8047285631381613


 80%|████████  | 4/5 [13:52<03:23, 203.80s/it]

mean epoch 4 error :  2.804739723305552


100%|██████████| 5/5 [16:52<00:00, 202.44s/it]

mean epoch 5 error :  2.8047457629941634


2nd batch

In [210]:
for e in tqdm(range(epochs)) :

    error = 0
    
    for x,y in zip(preprocess(batch2,len(batch2))[0],preprocess(batch2,len(batch2))[1]) :

        # Forward propagation :
        output = x

        for layer in image_bluring_layers : 
            output = layer.forward(output)

        for layer in model :
            
            output = layer.forward(output)
           
       
        loss = cross_entropy(output,y)
        error += loss.compute()
    
        # Backward propagation

        grad = loss.grad() 

        for layer in reversed(model) :          
   
            grad = layer.backward(grad,eta)


    error /= len(batch2)
    print(f"mean epoch {e+1} error : ", str(error)) 

  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [04:48<52:58, 288.94s/it]

mean epoch 1 error :  2.7901071558061155


 17%|█▋        | 2/12 [09:37<48:09, 288.91s/it]

mean epoch 2 error :  2.790107672570872


...

mean epoch 10 error :  2.790110076366203


 92%|█████████▏| 11/12 [52:57<04:48, 288.40s/it]

mean epoch 11 error :  2.7901102566740983


100%|██████████| 12/12 [57:45<00:00, 288.75s/it]

mean epoch 12 error :  2.7901104216394614


3rd batch

In [211]:
for e in tqdm(range(epochs)) :

    error = 0
    
    for x,y in zip(preprocess(batch3,len(batch3))[0],preprocess(batch3,len(batch3))[1]) :

        # Forward propagation :
        output = x

        for layer in image_bluring_layers : 
            output = layer.forward(output)

        for layer in model :
            
            output = layer.forward(output)
           
       
        loss = cross_entropy(output,y)
        error += loss.compute()
    
        # Backward propagation

        grad = loss.grad() 

        for layer in reversed(model) :          
   
            grad = layer.backward(grad,eta)


    error /= len(batch3)
    print(f"mean epoch {e+1} error : ", str(error)) 

  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [04:46<52:28, 286.24s/it]

mean epoch 1 error :  2.790110564905371


 17%|█▋        | 2/12 [09:32<47:43, 286.35s/it]

mean epoch 2 error :  2.7901107050930865


...

mean epoch 10 error :  2.7901115249222967


 92%|█████████▏| 11/12 [52:42<04:47, 287.32s/it]

mean epoch 11 error :  2.7901116006358664


100%|██████████| 12/12 [57:29<00:00, 287.47s/it]

mean epoch 12 error :  2.79011167207893


4th batch

In [212]:
for e in tqdm(range(epochs)) :

    error = 0
    
    for x,y in zip(preprocess(batch4,len(batch4))[0],preprocess(batch4,len(batch4))[1]) :

        # Forward propagation :
        output = x

        for layer in image_bluring_layers : 
            output = layer.forward(output)

        for layer in model :
            
            output = layer.forward(output)
           
       
        loss = cross_entropy(output,y)
        error += loss.compute()
    
        # Backward propagation

        grad = loss.grad() 

        for layer in reversed(model) :          
   
            grad = layer.backward(grad,eta)


    error /= len(batch4)
    print(f"mean epoch {e+1} error : ", str(error)) 

  8%|▊         | 1/12 [04:46<52:33, 286.70s/it]

mean epoch 1 error :  2.7901117404207474


 17%|█▋        | 2/12 [09:33<47:46, 286.62s/it]

mean epoch 2 error :  2.790111804262583


 25%|██▌       | 3/12 [14:20<43:02, 286.90s/it]

...

mean epoch 9 error :  2.7901121707770575


 83%|████████▎ | 10/12 [48:01<09:40, 290.25s/it]

mean epoch 10 error :  2.7901122139259718


 83%|████████▎ | 10/12 [48:41<09:44, 292.17s/it]


KeyboardInterrupt: 

5th batch

In [ ]:
for e in tqdm(range(epochs)) :

    error = 0
    
    for x,y in zip(preprocess(batch5,len(batch5))[0],preprocess(batch5,len(batch5))[1]) :

        # Forward propagation :
        output = x

        for layer in image_bluring_layers : 
            output = layer.forward(output)

        for layer in model :
            
            output = layer.forward(output)
           
       
        loss = cross_entropy(output,y)
        error += loss.compute()
    
        # Backward propagation

        grad = loss.grad() 

        for layer in reversed(model) :          
   
            grad = layer.backward(grad,eta)


    error /= len(batch5)
    print(f"mean epoch {e+1} error : ", str(error)) 

  5%|▌         | 1/20 [04:08<1:18:44, 248.67s/it]

mean epoch 1 error :  2.786221814778073


 10%|█         | 2/20 [08:18<1:14:43, 249.10s/it]

mean epoch 2 error :  2.7862218324719126


 15%|█▌        | 3/20 [12:27<1:10:31, 248.94s/it]

...

mean epoch 18 error :  2.78622206630101


 95%|█████████▌| 19/20 [1:18:58<04:09, 249.55s/it]

mean epoch 19 error :  2.7862220783970497


100%|██████████| 20/20 [1:23:07<00:00, 249.37s/it]

mean epoch 20 error :  2.786222090250353


Test of the network

In [ ]:
for x,y in zip(preprocess(test_set,150)[0],preprocess(test_set,150)[1]) :

    output = x
    for layer in image_bluring_layers : 
            output = layer.forward(output)
    good_fit = 0
    for layer in model :

        output = layer.forward(output)
    
    print(f"pred : {np.argmax(output)}, proba : {np.max(output)}, true : {np.argmax(y)}")
    if np.argmax(output) -1 == np.argmax(y) : 
        good_fit += 1 

print(f"accuracy : {good_fit/len(test_set)}")

pred : 12, proba : 0.0763146645063138, true : 8
pred : 12, proba : 0.07631466433628721, true : 7
pred : 12, proba : 0.07631466318349256, true : 13
pred : 12, proba : 0.07631466241877055, true : 6
pred : 12, proba : 0.07631466494743243, true : 2
pred : 12, proba : 0.07631466424011488, true : 13
pred : 12, proba : 0.07631466416210957, true : 14
pred : 12, proba : 0.07631466438317891, true : 2
pred : 12, proba : 0.07631466376835683, true : 6
pred : 12, proba : 0.07631466322775185, true : 2
pred : 12, proba : 0.07631466478838489, true : 8
pred : 12, proba : 0.07631466434737495, true : 5
pred : 12, proba : 0.07631466455300882, true : 10
pred : 12, proba : 0.07631466533065609, true : 7
pred : 10, proba : 0.07631466358090115, true : 8
pred : 12, proba : 0.07631466414253388, true : 0
pred : 12, proba : 0.07631466453416688, true : 14
pred : 10, proba : 0.07631466618769736, true : 3
pred : 12, proba : 0.07631466364069436, true : 0
pred : 12, proba : 0.07631466468361497, true : 4
pred : 12, proba